In [1]:
!pip install -q newspaper3k==0.2.8 python-dotenv

In [2]:
!pip install -U langchain-community

In [3]:
!pip install -U langchain-openai

In [4]:
import json
import os
from dotenv import load_dotenv
import requests
from newspaper import Article
from langchain_core.messages import HumanMessage # Updated import
from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from google.colab import userdata
load_dotenv()

False

In [5]:
openai_api_key = userdata.get('OPENAI_API_KEY').strip()
os.environ['OPENAI_API_KEY'] = openai_api_key

In [6]:
headers ={'User-Agent': '''Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'''}

article_url ="""https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"""

session =requests.Session()

try:
  response =session.get(article_url, headers=headers, timeout=10)

  if response.status_code == 200:
    article = Article(article_url)
    article.download()
    article.parse()
    print(f"Title:{article.title}")
    print(f"Text: {article.text}")
  else:
    print(f"Status code:{response.status_code}")
    print(f"Failed to fetch article at {article_url}")
except Exception as e:
  print(e)

Title:Meta claims its new AI supercomputer will set records
Text: Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and the aim is for it to be capable of training models with trillions of parameters.

“We hope RSC will help us build entirely new AI systems that can, for example, power real-time voice translations to large groups of people, each speaking a different language, so they can seamlessly collaborate on a research project or play an AR game together,” wrote Meta in a blog post.

“Ultimately, the work done with RSC will pave the way toward building technologies for the next major computing 

In [7]:
template ="""
As an advanced AI, you've been tasked to summarize online articles into bulleted points.
Here are a few examples of how you've done this in the past:
Example 1:
Original Article: 'The Effects of Climate Change
Summary:
-Climate change is causing a rise in global temperatures.
-This leads to melting ice caps and rising sea levels.
-Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial IntelligenceSummary:
-Artificial Intelligence (AI) has developed significantly over the past decade.
-AI is now used in multiple fields such as healthcare, finance, and transportation.
-The future of AI is promising but requires careful regulation.Now, here's the article you need to summarize:
==================
Title: {article_title}{article_text}
==================
Please provide a summarized version of the article in a bulleted list format."""

In [8]:
# Format the Prompt
prompt =template.format(article_title=article.title, article_text=article.text)
messages =[HumanMessage(content=prompt)]

In [9]:
from langchain_openai import ChatOpenAI

# load the model
chat =ChatOpenAI(model_name="gpt-4-turbo", temperature=0.0, openai_api_key=os.environ['OPENAI_API_KEY'])

# generate summary
summary =chat.invoke(messages)
print(summary.content)

- Meta (formerly Facebook) has announced the development of a new AI supercomputer named the AI Research SuperCluster (RSC).
- The RSC is not yet complete but is expected to be the world's fastest supercomputer by mid-2022.
- It is designed for training extensive natural language processing (NLP) and computer vision models.
- Meta aims to use the RSC for advanced applications such as real-time voice translations for large, multilingual groups, potentially enhancing collaborative projects and augmented reality (AR) gaming.
- The RSC will significantly increase computational speeds, being 20x faster than Meta's current V100-based clusters and improving large-scale NLP workflow training by 3x.
- Training times for models with tens of billions of parameters will reduce from nine weeks to three weeks.
- Unlike previous systems, the RSC incorporates enhanced security and privacy controls, allowing the use of real-world data from Meta's production systems for training purposes.
- This capabil

In [10]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List

In [11]:
class ArticleSummary(BaseModel):
  title: str = Field(description="Title of the article")
  summary: List[str] = Field(description="""Bulleted list summary of the article""")

  @validator('summary',allow_reuse=True)
  def has_three_or_more_line(cls,list_of_lines):
    if len(list_of_lines) < 3:
      raise ValueError("""Generated summary has less than three bullet points!""")
    return list_of_lines

#set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

/tmp/ipython-input-3922893073.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('summary',allow_reuse=True)
/tmp/ipython-input-3922893073.py:5: PydanticDeprecatedSince20: `allow_reuse` is deprecated and will be ignored; it should no longer be necessary. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('summary',allow_reuse=True)


In [13]:
from langchain_core.prompts import PromptTemplate

template ="""

You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.


==================

Title: {article_title}

{article_text}


{format_instructions}


"""

prompt_template = PromptTemplate(
    template = template,
    input_variables=["article_title","article_text"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

# Create a chain using LCEL
chain = prompt_template | model | parser

# generate summary
output = chain.invoke({"article_title": article.title, "article_text": article.text})

print(output)

title='Meta claims its new AI supercomputer will set records' summary=["Meta (formerly Facebook) unveils AI supercomputer called AI Research SuperCluster (RSC) aiming to be the world's fastest", 'RSC to be fully built by mid-2022, capable of training models with trillions of parameters', 'Meta hopes RSC will enable new AI systems for real-time voice translations and metaverse technologies', 'Expected to be 20x faster than current clusters, 9x faster at running NVIDIA Collective Communication Library, and 3x faster at training NLP workflows', 'RSC designed with security and privacy controls to use real-world data for research tasks like identifying harmful content on platforms']
